## 总结DataFrame的级联

pd.concat()  将两个表进行连接
- objs 多个DataFrame之间的连接， 一般用tuple表示
- axis 坐标轴
    - 0 行（数据对象）连接
    - 1 列 (特征之间) 连接
- keys 多层索引
    - 指定连接表的一级索引, 如两个表连接，应该提供两个表的一级索引
- join 连接方式， 类似于数据库的连接
    - outer 外连接， 保留连接后的全部数据, 并集
    - inner 内连接， 保留两个表之间相同的部分， 交集
- join_axes 保留连接之后的行索引或列
    - 必须使用Index封装索引或列的标签
    - 数据是 [] list格式， list中存放是 Index对象
    - Index(['标签1', '标签2'])
    
df.append(df2)  将df2的数据以行方式追加到df表中

# DataFrame Nan值的处理
- df.isnull()  查找df表中存在NaN的bool标识
   
   返回的结果还是一个DataFrame，只不过每一个特征的数据都变为True或False
    - any() 在行或列上，存在任一个True时， 按axis提取数据
        - axis 0 每行之间的相同特征之间比较, 针对的特征
        - axis 1 某一行所有特征之间的比较， 针对的数据对象
    - all() 在行或列上， 全部是True时， 按axis提取数据
- df.notnull()

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame, Index, MultiIndex

In [7]:
df = DataFrame([
    [1, None, 4],
    [2, 11, 9],
    [5, 18, 10],
], columns=('start','mid', 'end'))
df

,start,mid,end
0,1,NaN,4
1,2,11.0,9
2,5,18.0,10


In [8]:
df.isnull().any(axis=0)

start    False
mid       True
end      False
dtype: bool

In [9]:
# 查看包含Nan值的列数据
df.loc[:, df.isnull().any(axis=0)]

,mid
0,NaN
1,11.0
2,18.0


In [11]:
df.loc[0] = None
df

,start,mid,end
0,NaN,NaN,NaN
1,2.0,11.0,9.0
2,5.0,18.0,10.0


In [12]:
# 查看特征全部为Nan的某一人
df.loc[df.isnull().all(axis=1)]

,start,mid,end
0,NaN,NaN,NaN


In [20]:
# df.loc[:, 1] = None
# df.drop(1, axis=1, inplace=True)  # 删除指定行或列的标签
# df.loc[:,'mid'] = None
df.loc[:,'mid'] = np.nan
df

,start,mid,end
0,NaN,NaN,NaN
1,2.0,NaN,9.0
2,5.0,NaN,10.0


In [21]:
# 查看所有的人某一特征全为Nan
df.loc[:, df.isnull().all(axis=0)]

,mid
0,NaN
1,NaN
2,NaN


# merge合并操作
- 不同关系的合并
    - 一对一情况
    - 一对多的情况
    - 多对多的情况

In [23]:
a = DataFrame([
    ['张飞', 20],
    ['关羽', 21],
    ['刘备', 24],
    ['赵云', 19]
], columns=('name', 'age'))

b = DataFrame([
    ['张飞', '男'],
    ['关羽', '男'],
    ['黄忠', '男'],
    ['花木兰', '女']
], columns=('name', 'sex'))

display(a, b)

,name,age
0,张飞,20
1,关羽,21
2,刘备,24
3,赵云,19


,name,sex
0,张飞,男
1,关羽,男
2,黄忠,男
3,花木兰,女


In [24]:
pd.merge(a, b) # 内合并

,name,age,sex
0,张飞,20,男
1,关羽,21,男


- 一对多的合并

In [26]:
# 多端的数据表
c = DataFrame([
    ['张飞', '东汉'],
    ['张飞', '蜀汉'],
    ['关羽', '东汉'],
    ['关羽', '蜀汉'],
    ['黄忠', '东汉'],
    ['黄忠', '蜀汉'],
    ['花木兰', '北魏']
], columns=('name', 'country'))
pd.merge(a, c)  # 一对多合并

,name,age,country
0,张飞,20,东汉
1,张飞,20,蜀汉
2,关羽,21,东汉
3,关羽,21,蜀汉


- 多对多的合并

In [27]:
d = DataFrame([
    ['张飞', '徐州'],
    ['张飞', '新野'],
    ['张飞', '成都'],
    ['关羽', '徐州'],
    ['关羽', '新野'],
    ['关羽', '荆州'],
    ['黄忠', '长沙'],
    ['黄忠', '荆州'],
    ['黄忠', '成都'],
    ['花木兰', '洛阳']
], columns=('name', 'city'))

pd.merge(c, d)

,name,country,city
0,张飞,东汉,徐州
1,张飞,东汉,新野
2,张飞,东汉,成都
3,张飞,蜀汉,徐州
4,张飞,蜀汉,新野
5,张飞,蜀汉,成都
6,关羽,东汉,徐州
7,关羽,东汉,新野
8,关羽,东汉,荆州
9,关羽,蜀汉,徐州


## key的规范化
- on  如果两个表中存在多个列标签相同时，可以通过on来指定哪一个标签合并
- left_on 如果两个表中标签名不同，可以指定左表的合并的列标签
- right_on 同上， 可以指定右表的合并的列标签

In [34]:
# df.rename(index, columns) 可以重新命名行索引和列标签
#   要求： 字典格式， key 原名， value 新名
#   说明： inplace=True， 在原表上修改， 为False返回一个副本copy
a1 = a.rename(columns={'name':'姓名'})
# a1.drop('name', axis=1, inplace=True)  # 删除列
a1

,姓名,age
0,张飞,20
1,关羽,21
2,刘备,24
3,赵云,19


In [36]:
pd.merge(a1, b,
         left_on='姓名',
         right_on='name').drop('姓名', axis=1)

,age,name,sex
0,20,张飞,男
1,21,关羽,男


In [38]:
df1 = DataFrame([
    ['东方不败', '123', 1200],
    ['张三丰', '300', 2200],
    ['令狐冲', '800', 9200],
    ['岳灵珊', '110', 10000],
], columns=('name', 'phone','salary'))

df2 = DataFrame([
    ['张三丰', '301', '2017'],
    ['令狐冲', '801', '2017'],
    ['岳灵珊', '110', '2018'],
    ['萧峰', '120', '2016'],
    ['虚竹', '114', '2019'],
], columns=('name', 'phone','hire_date'))

pd.merge(df1, df2)  # 默认情况，所有相同的列值都必须一样，内合并

,name,phone,salary,hire_date
0,岳灵珊,110,10000,2018


In [39]:
# 通过 on 指定哪一相同的列进行连接
pd.merge(df1, df2, on='name')

,name,phone_x,salary,phone_y,hire_date
0,张三丰,300,2200,301,2017
1,令狐冲,800,9200,801,2017
2,岳灵珊,110,10000,110,2018


In [42]:
# 两表中其余相同的列，如果不参与合并，默认[_x, _y]方式增加后辍名
# suffixes 指定左右两表中相同列的后辍名
# pd.merge(df1, df2,
#          on=['name','phone'], suffixes=['-家庭', '-工作'])
pd.merge(df1, df2,
         on='name', suffixes=['-家庭', '-工作'])

,name,phone-家庭,salary,phone-工作,hire_date
0,张三丰,300,2200,301,2017
1,令狐冲,800,9200,801,2017
2,岳灵珊,110,10000,110,2018


## 内外合并
- how {'inner', 'outer', 'left', 'right'}
- 默认 inner, 内合并
- outer , 外合并， 将两个中所有数据全部显示，会出现NaN值
- left ,  左外合并，除相同的数据之外，还会显示左表的其余数据
- right,  右外合并，除相同的数据之外，还会显示右表的其余数据

In [46]:
pd.merge(a, b, how='outer').fillna(method='ffill', axis=0)

,name,age,sex
0,张飞,20.0,男
1,关羽,21.0,男
2,刘备,24.0,男
3,赵云,19.0,男
4,黄忠,19.0,男
5,花木兰,19.0,女


In [47]:
pd.merge(a, b, how='left') 

,name,age,sex
0,张飞,20,男
1,关羽,21,男
2,刘备,24,NaN
3,赵云,19,NaN


In [48]:
pd.merge(a, b, how='right') 

,name,age,sex
0,张飞,20.0,男
1,关羽,21.0,男
2,黄忠,NaN,男
3,花木兰,NaN,女


## Series的unique() 去重

In [49]:
ddd = pd.merge(a, b, how='right').fillna(method='ffill', axis=0)
ddd

,name,age,sex
0,张飞,20.0,男
1,关羽,21.0,男
2,黄忠,21.0,男
3,花木兰,21.0,女


In [51]:
ddd['age'].unique()

array([ 20.,  21.])

In [52]:
ddd['sex'].unique()

array(['男', '女'], dtype=object)

## DataFrame的查询
- df.query('查询字符串')
- == 等值条件
- & 与条件
- | 或条件

In [61]:
ddd.iloc[-1, 1] = 20
ddd

,name,age,sex
0,张飞,20.0,男
1,关羽,21.0,男
2,黄忠,21.0,男
3,花木兰,20.0,女


In [63]:
ddd.query('age == 21 | sex == "女"')

,name,age,sex
1,关羽,21.0,男
2,黄忠,21.0,男


In [66]:
ccc = pd.merge(a, b, how='right')
ccc

,name,age,sex
0,张飞,20.0,男
1,关羽,21.0,男
2,黄忠,NaN,男
3,花木兰,NaN,女
